# Iterative Quantum Amplitud Estimation (IQAE) module

Present notebook reviews the **Iterative Quantum Amplitude Estimation** (**IQAE**) algorihtm. 

**BE AWARE** this algorithm is different from the **Iterative Quantum Phase Estimation** (**IQPE**). The second one is an algorithm for pure *phase estimation* of an unitary operator meanwhile the first one is an algorihtm for direct solving of **Amplitude Estimation** problem based on the *amplification* capabilities of a Grover operator. 

The **IQAE** algorithm was implemented into the module *iterative_quantum_ae* of the package *AE* of library *QQuantLib* (**QQuantLib/AE/iterative_quantum_ae.py**). This algorithm was developed as a python class called: *IQAE*

Present notebook and modules are based on the following references:

* *Grinko, D., Gacon, J., Zoufal, C. & Woerner, S.*. Iterative Quantum Amplitude Estimation. npj Quantum Information 7, 2021. https://www.nature.com/articles/s41534-021-00379-1

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
x = np.arange(2**n)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. IQAE algorithm.

### 2.1 The Amplitude Estimation Problem

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{O}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.  We can define an associated angle to $\sqrt{a}$ as $\sin^2{\theta} = a$, and the problem is thus rewritten as:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$

The foundations of any amplitude estimation algorithm is the grover operator $\mathcal{Q}$, built onto the oracle $\mathcal{O}$, that has the following effect over our state $|\Psi\rangle$:

$$\mathcal{G}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$

for more information about the grover operator and the amplitude amplification algorithm check the notebook **02_AmplitudeAmplification_Operators.ipynb**.




### 2.2 IQAE algoritm summary

Given an error $\epsilon$ and a confident interval $\alpha$, the **IAQE** algorithm allows to estimate $(\theta_l, \theta_u)$ such that the $\theta$ angle of the Amplitude Estimation problem satisfies that:

$$P\big[\theta \in [\theta_l, \theta_u]\big] \gt 1-\alpha$$

and

$$\frac{\theta_u-\theta_l}{2} \leq \epsilon$$


This result can be extended in a straightforward way to $a=\sin^2{\theta}$, so the **IQPE** algorithm will provide $(a_l, a_u)$ that satisfies:

$$P\big[a \in [a_l, a_u]\big] \gt 1-\alpha$$

and

$$\frac{a_u-a_l}{2} \leq \epsilon$$


### 2.3 Creating object from IQAE class

We have implemented and python class called **IQAE** into the **QQuantLib/AE/iterative_quantum_ae** module that allows us to use the **IQAE** algorithm.

For creating the **IQAE** class the conventions used in **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae.py** module should be followed: 

We have some mandatory inputs:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binnary representation as a python list
3. index: list of the qbits affected by the Grover operator.

And some optional inputs, used for algorithm configuration, that can be given as a python dictionary:
* qpu: QLM solver tha will be used
* epsilon ($\epsilon$): the precision. Ensures that the width of the interval is (see Section 2.2), at most, $2\epsilon$ (default: 0.01).
* alpha ($\alpha$): the accuracy. Ensures that the probability of $a$ not laying within the given interval (see Section 2.2) is, at most, $\alpha$ (default: 0.05).
* shots: the number of shots on each iteration of the algorithm (default: 100).


In [ ]:
#import the class
from QQuantLib.AE.iterative_quantum_ae import IQAE

For showing how our class and the algorithm works, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.



In [ ]:
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

print('Real Value of a: ', a)

epsilon = 0.001
shots = 100
alpha = 0.05


In [ ]:
iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

When the class is created the based oracle Grover operator is created too and can be acces using the **\_grover_oracle**  property of the class

In [ ]:
c=iqae._grover_oracle
%qatdisplay c --svg --depth 2

### 2.4 IQAE algorithm scheme.


As explained, the inputs for the **IQAE** algorihtm are:

* Error in the estimation of the angle $\theta$: $\epsilon$.
* Confidence interval for the $\theta$: $\alpha$.

The main steps of the **IQAE** algorithm, in a simplified way, are:

1. The algorithm initialize the limits for the angle to estimation, $\theta$, to $[\theta_l, \theta_u] = [0, \frac{\pi}{2}]$.
2. The algorithm calculates the maximum number of iterations $T$ that should be necesary in order to satisfy the error estimation $\epsilon$:
    * $T(\epsilon) \in \mathcal{N} \; / \;T(\epsilon) \geq \log_2(\frac{\pi}{8\epsilon})$
    * In the framework of the **IQAE** algorithm an iteration is a selection of a different integer $k$

4. Selection of $k$ in a algorithm iteration. **This is the critical routine of the algorithm**: the routine tries to obtain the biggest $k$ (until some fixed limit) that ensures that $(4*k+2)\theta_l$ and $(4*k+2)\theta_u$ are contained totally in the $[0,\pi]$ or the $[\pi, 2\pi]$ semiplane. If this is obtained the selection routine will return the $k$ and the semiplane.
5. For a selected $k$ the **IQAE** algorithm creates the corresponding circuit for doing:
    * $$\mathcal{G}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$

6. Using $N$ shots compute the probability $a_k$ of obtaing the $|\Psi_0\rangle$ that will be:

$$P(|\Psi_0\rangle, k) = \sin^2((2*+1)\theta) = \frac{1-\cos((4k+2)\theta)}{2}=a_k$$

7. Using the number of measurements $N$, $T$ and $\alpha$ the algorithm calculates $\epsilon_{a_k}$ using:

    * $\epsilon_{a_{k}} = \sqrt{\frac{1}{2N}\log(\frac{2T}{\alpha})}$

8. Using the $\epsilon_{a_{k}}$ the algorithm computes some limits for $a_k$: $a_{k}^{min}$ and $a_{k}^{max}$

9. The algorithm computes $\theta_{k}^{min}$ and  $\theta_{k}^{max}$ from $a_{k}^{min}$ and $a_{k}^{max}$, using:
    * $a_k = \frac{1-\cos((4k+2)\theta)}{2}$
    * and the fact that $a_{k}^{min}$ and $a_{k}^{max}$ should be in one the semiplanes: $[0,\pi]$ or the $[\pi, 2\pi]$ (this is given by the selection routine of step 3)

10. Updating $\theta_l$ and $\theta_u$ using $\theta_{k}^{min}$ and $\theta_{k}^{max}$ respectively and the fact that the rotation due to $k$ aplication of Grover algorithm is $(4k+2)\theta$

At the end of each iteration $\theta_l-\theta_u$ is lower than at the begining. When $\theta_u-\theta_l \leq 2\epsilon$ the algorithm stops. 

**NOTE**

1. For ensure that $\theta_u-\theta_l \leq 2\epsilon$ is necesary that the number if iterations should be at most equal to T ($T(\epsilon) \geq \log_2(\frac{\pi}{8\epsilon})$).
2. For ensure that $P\big[\theta \in [\theta_l, \theta_u]\big] \gt 1-\alpha$ is mandatory that the error of each iteration should be: $\epsilon_{a_{k}} = \sqrt{\frac{1}{2N}\log(\frac{2T}{\alpha})}$



### 2.5 Example of IQAE workflow

Section 2.4 present a simple plot of the **IQAE** algorithm scheme. In present section we show an example of how this scheme is used for getting some intuition of how the **IQAE** algorihtm works. We will split the algorithm in 3 steps:

* Initialization.
* First iteration with $k=0$.
* Next iterations with $k\geq 0$

#### 2.5.1 Initialization

We need to do the initialization of the algorithm (setting the initial $\theta_l$, $\theta_u$) and getting the maximum number of iterations from $\epsilon$ ($T(\epsilon)$)

In [ ]:
#Initialization of IQAE

[theta_l,theta_u] = [0.0,np.pi/2]
k=0
#True for semiplane [0,pi]
flag = True
#Number of shots
shots = 100
#Number of iterations
T = int(np.ceil(np.log2(np.pi/(8*iqae.epsilon)))+1)

print('Max number of IQAE iterations: ',T)


#### 2.5.2 First iteration with $k=0$.

In the first iteration we are going to set $k=0$. Then we execute the complet iteration workflow:

In [ ]:
#First step
N=shots
print('#################### First Iteration k= {}. Start #################'.format(k))
print('k = ', k)
K = 4*k+2
print('K = 4*k+2= ', K)
DeltaTheta_initial = np.abs(theta_u-theta_l)
print('Creating the Quantum circuit with k= ', k)
routine = qlm.QRoutine()
wires = routine.new_wires(iqae.oracle.arity)
routine.apply(iqae.oracle,wires)
for j in range(k):
    routine.apply(iqae._grover_oracle,wires)
    
print('Computing the probabiliy of measure |Phi_0>')
results,_,_,_ = get_results(
    routine,linalg_qpu = linalg_qpu,
    shots = N,
    qubits = iqae.index
)
#Probability of measure |Phi_0>
a = results['Probability'].iloc[bitfield_to_int(iqae.target)]
print('probabiliy of measure |Phi_0> for {}: {} (a)'.format(k, a))
#Getting the error for a
epsilon_a = iqae.chebysev_bound(N,alpha/T)
print('epsilon for iteration {}: {}'.format(k, epsilon_a))
#using epsilon we compute new a limits
a_max = np.minimum(a+epsilon_a,1.0)
a_min = np.maximum(a-epsilon_a,0.0)
#getting theta_min and theta_min from a_min,a_max
[theta_min,theta_max] = iqae.invert_sector(a_min,a_max,flag)
#Updating theta_l and theta_u from theta_min,theta_max and K
theta_l = (2*np.pi*np.floor(K*theta_l/(2*np.pi))+theta_min)/K
theta_u = (2*np.pi*np.floor(K*theta_u/(2*np.pi))+theta_max)/K
print('New: [theta_l, theta_u]= [{}, {}]'.format(theta_l, theta_u))
DeltaTheta_present = np.abs(theta_u-theta_l)
print('#################### First Iteration k= {}. End #################'.format(k))

Now we compare the difference between the olds and the new $\theta_u$ and $\theta_l$

In [ ]:
print('Initial Delta Theta: ', DeltaTheta_initial)
print('Final Delta Theta: ', DeltaTheta_present)

As can be seen the difference now is lower

#### 2.5.3 Next iterations with $k\geq 0$

In the next iterations the first step is getting the $k$ for the correspondient iteration. As explained in step 3 of Section 2.3 this is the **critical** routine of the algorithm. This **routine** will use the currents $\theta_l$, $\theta_u$ and the before step $k$ for computing the biggest $k$ (until some limit) that will ensure that $(4*k+2)\theta_l$ and $(4*k+2)\theta_u$ are contained totally in the $[0,\pi]$ or the $[\pi, 2\pi]$ semiplane. This is done by the *find_next_k* method of the class. This method need as input:

* k: $k$ of the before iteration
* theta_lower: $\theta_l$
* theta_upper: $\theta_u$
* flag: flag for keeping the track of the semiplane (True for $[0, \pi]$)
* r: parameter of the routine (default 2). 

The outputs of the method will be:
* k: the new $k$ for the current iteration.
* flag: semiplane where $(4*k+2)\theta_l$ and $(4*k+2)\theta_u$ will be contained (True for $[0, \pi]$)
 
For executing the complete iteration execute the following cell

In [ ]:
print('Searching for the new k')
[k,flag] = iqae.find_next_k(k,theta_l,theta_u,flag)

print('#################### ITERATION with k = {}. Start #################'.format(k))
print('New k= ', k)
K = 4*k+2
print('New K= 4*k+2= ', K)

DeltaTheta_initial = np.abs(theta_u-theta_l)

print('Creating the Quantum circuit with k= ', k)
routine = qlm.QRoutine()
wires = routine.new_wires(iqae.oracle.arity)
routine.apply(iqae.oracle,wires)
for j in range(k):
    routine.apply(iqae._grover_oracle,wires)
    
print('Computing the probabiliy of measure |Phi_0>')    
results,_,_,_ = get_results(
    routine,linalg_qpu = linalg_qpu,
    shots = N,
    qubits = iqae.index
)
#Probability of measure |Phi_0>
a = results['Probability'].iloc[bitfield_to_int(iqae.target)]
print('probabiliy of measure |Phi_0> for {}: {} (a)'.format(k, a))

#Getting the error for a
epsilon_a = iqae.chebysev_bound(N,alpha/T)
print('epsilon for iteration {}: {}'.format(k, epsilon_a))
#using epsilon we compute new a limits
a_max = np.minimum(a+epsilon_a,1.0)
a_min = np.maximum(a-epsilon_a,0.0)
#getting theta_min and theta_min from a_min,a_max
[theta_min,theta_max] = iqae.invert_sector(a_min,a_max,flag)
#Updating theta_l and theta_u from theta_min,theta_max and K
theta_l = (2*np.pi*np.floor(K*theta_l/(2*np.pi))+theta_min)/K
theta_u = (2*np.pi*np.floor(K*theta_u/(2*np.pi))+theta_max)/K
print('New: [theta_l, theta_u]= [{}, {}]'.format(theta_l, theta_u))
DeltaTheta_present = np.abs(theta_u-theta_l)

print('#################### ITERATION with k = {}. End #################'.format(k))
print('Initial Delta Theta: ', DeltaTheta_initial)
print('Final Delta Theta: ', DeltaTheta_present)

In order to do several iterations execute the cell several times.

In [ ]:
print('Is enough: ', DeltaTheta_present < iqae.epsilon)

Sometimes the routine for finding the new $k$ cannot get a propper new $k$, then the old $k$ is used again. In order to avoid repeat the same $k$ a lot of times we can accumulate the measurements done for one $k$ and using them for calculating the step error $\epsilon_{a_{k}}$

## 3. IQAE complete execution.

In section 2.4 the basic scheme of the **IQAE** algorithm was plotted for pedagogical purposes. The **IQAE** class deals with the code presented in section 2.4 (in fact implement some improvements for getting a better performance) in transparent way. It is expected that the user of the **IQAE** class only executes following methods:

* *iqae* method.
* *run* method
* *display_information* method

### 3.1 The *iqae* method

In order to execute the complete algorithm using the **IQAE** class the *iqae* method is used. This method has the following inputs:

* epsilon ($\epsilon$): error in the estimation of the angle $\theta$ (default: 0.01).
* shots: number of shots for the measurement of the circuit ($N_{shots}$ (default: 100).
* alpha ($\alpha$): confidence interval for the $\theta$ (default: 0.05).

This method returns the limits for the $a$ estimation: $(a_{\min},a_{\max})$

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

epsilon = 0.001
shots = 100
alpha = 0.05

iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

In [ ]:
epsilon_t = 0.001
[a_l, a_u]=iqae.iqae(
    epsilon = epsilon_t,
    shots = 500,
    alpha=0.01
)

In [ ]:
print('Bounds for a: [a_l, a_u] = [{}, {}]'.format(a_l, a_u))
a_estimated = (a_u+a_l)/2.0
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a)
print('|a_l-a_estimated| = ', np.abs(a_estimated-a))
print('Error estimation wanted: ', epsilon_t)

### 3.2 *display_information* method

The display method gives some information of the inner working of the **IQAE** algorithm. The inputs are the same that for the *iqae* methdo.

In [ ]:
iqae.display_information(
    epsilon = iqae.epsilon,
    shots= iqae.shots, 
    alpha = iqae.alpha
)

### 3.3 The *run* method

Finally a *run* method for a direct implementation of the **IQAE** algorithm was implemented. In this case the user can configure all the properties of the **IQAE** class and the *run* method will execute the method using the fixed attributes of the class. Finally the method returns the estimation of $a=\frac{a_u+a_l}{2}$. Additionally the *run* method populates following class attributes:

* ae_l: the lower limit for a $a_l$.
* ae_l: the upper limit for a $a_u$.
* theta_l: the lower limit for $\theta$: $theta_l$.
* theta_u: the upper limit for $\theta$: $theta_u$.
* ae: the amplitude estimation parameter computing as: $a=\frac{a_u+a_l}{2}$
* theta: the estimated $\theta=\frac{\theta_u+\theta_l}{2}$

In [ ]:
#First we create the class
target = [0,0,1]
index = [0,1,2]
a = probability[bitfield_to_int(target)]

epsilon = 0.001
shots = 100
alpha = 0.05

iqae_dict = {
    'epsilon': epsilon,
    'shots': shots,
    'alpha': alpha,
    'qpu': linalg_qpu
}

iqae = IQAE(oracle, target = target, index = [0,1,2], **iqae_dict)

In [ ]:
a_estimated = iqae.run()

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a)
print('Bounds for a: [iqae.ae_l, iqae.ae_u] = [{}, {}]'.format(iqae.ae_l, iqae.ae_u))
print('Bounds for theta: [iqae.theta_l, iqae.theta_u] = [{}, {}]'.format(iqae.theta_l, iqae.theta_u))
print('Estimated theta: iqae.theta = ', iqae.theta)
print('Estimated a: iqae.ae = ', iqae.ae)

In [ ]:
print(' a_real-iqae.ae: ', abs(iqae.ae-a))
print('Epsilon: ', iqae.epsilon)
print('iqae error: ', iqae.ae_u-iqae.ae_l)